In [1]:
from pathlib import Path

import pandas as pd

from src.calculate_pairs import generate_complete_sets
from src.csv_writer import write_to_csv
from src.image_maker import draw_splines_on_circular_numbers
from src.sort_overlapping_lists import sort_lists_smart

In [4]:
n = 60
n_solutions = 200

## Generate complete sets of square-sum pairs for numbers 1..n

In [30]:
all_sets = generate_complete_sets(n_numbers=n)
write_to_csv(all_sets, output_csv=f"data/complete_sets_n{n}.csv")

Found 30 pairs of numbers that sum to a perfect square, between 1 and 24.
Building adjacency graph...
Graph built. Number 1 has 4 possible partners.


Adaptive Parallelization Strategy
Available CPU cores: 11
Splitting depth chosen: Level 3
Estimated branches: 36
Target branches: ~16

Strategy: Level 3 Splitting
  - Generated 36 3-level branch configurations
  - Creating 11 workers
  - Each worker explores: (1, p1), (n, p2), (m, p3) → full search

Starting parallel search with 11 workers...

✓ Worker 1 finished: 0 solutions, 56 attempts
✓ Worker 1 finished: 0 solutions, 100 attempts
✓ Worker 1 finished: 0 solutions, 33 attempts
✓ Worker 1 finished: 0 solutions, 53 attempts
✓ Worker 2 finished: 0 solutions, 35 attempts
✓ Worker 2 finished: 0 solutions, 69 attempts
✓ Worker 2 finished: 0 solutions, 79 attempts
✓ Worker 2 finished: 0 solutions, 33 attempts
✓ Worker 3 finished: 0 solutions, 31 attempts
✓ Worker 3 finished: 0 solutions, 50 attempts
✓ Worker 3 finished: 0 solutions, 46 attem

## Generate ordered sets of square-sum pairs with minimal change between consecutive sets

In [38]:
def parse_pair(pair_str: str) -> tuple[int, int]:
    a, b = map(int, pair_str.split("-"))
    return a, b

In [ ]:
# Load first n_solutions from CSV
all_sets = (
    pd.read_csv(
        f"data/complete_sets_n{n}.csv",
        nrows=200,
    )
    .drop(columns=["Pair No"])
    .map(parse_pair)
    .values.tolist()
)

ordered = sort_lists_smart(all_sets)

write_to_csv(ordered, output_csv=f"data/ordered_sets_n{n}.csv")

Sorting 200 lists using method: greedy_best

All complete sets have been written to 'data/ordered_sets_n60.csv'.


## Generate image for each set of square-sum pairs

In [5]:
max_images = n_solutions + 1

# Read from CSV
try:
    df_iterator = pd.read_csv(f"data/ordered_sets_n{n}.csv", chunksize=100)
except FileNotFoundError:
    df_iterator = pd.read_csv(f"data/complete_sets_n{n}.csv", chunksize=100)

# Clear images directory
images_dir = Path("images")

# Create directory for n
n_dir = images_dir / f"n{n}"
n_dir.mkdir(parents=True, exist_ok=True)


if n_dir.exists() and n_dir.is_dir():
    for file in n_dir.iterdir():
        if file.is_file():
            file.unlink()

images_processed = 0
for df in df_iterator:
    for index, row in df.iterrows():
        # Extract pairs from the row (assuming pairs are in the format "a-b").
        # Skip the first column which has Pair No.
        pairs = []
        for col in df.columns[1:]:
            pair_str = row[col]
            if pd.isna(pair_str):
                continue  # Skip if the cell is empty
            a, b = map(int, pair_str.split("-"))
            pairs.append((a, b))

        images_processed += 1
        # print(f"Processing image {images_processed} for n={n}, index={index}")
        draw_splines_on_circular_numbers(n, pairs, f"images/n{n}/{index:06}.png")
        if images_processed >= max_images:
            break
    if images_processed >= max_images:
        break

Image with splines saved to images/n60/000000.png
Image with splines saved to images/n60/000001.png
Image with splines saved to images/n60/000002.png
Image with splines saved to images/n60/000003.png
Image with splines saved to images/n60/000004.png
Image with splines saved to images/n60/000005.png
Image with splines saved to images/n60/000006.png
Image with splines saved to images/n60/000007.png
Image with splines saved to images/n60/000008.png
Image with splines saved to images/n60/000009.png
Image with splines saved to images/n60/000010.png
Image with splines saved to images/n60/000011.png
Image with splines saved to images/n60/000012.png
Image with splines saved to images/n60/000013.png
Image with splines saved to images/n60/000014.png
Image with splines saved to images/n60/000015.png
Image with splines saved to images/n60/000016.png
Image with splines saved to images/n60/000017.png
Image with splines saved to images/n60/000018.png
Image with splines saved to images/n60/000019.png


## Combines images into a video

In [6]:
from src.video_maker import create_video_from_images, create_video_with_transitions

# # # Basic usage
# create_video_from_images(
#     image_folder=f"images/n{n}/",
#     output_video=f"video/n{n}.mp4",
#     fps=30,
#     frames_per_image=5,
# )


# Basic usage - smooth transitions between images
create_video_with_transitions(
    image_folder=f"images/n{n}/",
    output_video=f"video/n{n}.mp4",
    fps=30,
    frames_per_image=10,
    transition_frames=3,
)

Found 200 images
Video settings: 30 FPS, 10 frames per image, 3 transition frames
Total video duration: 86.57 seconds (1.44 minutes)
Video dimensions: 1785x1785
Processing image 1/200: 000000.png
Processing image 2/200: 000001.png
Processing image 3/200: 000002.png
Processing image 4/200: 000003.png
Processing image 5/200: 000004.png
Processing image 6/200: 000005.png
Processing image 7/200: 000006.png
Processing image 8/200: 000007.png
Processing image 9/200: 000008.png
Processing image 10/200: 000009.png
Processing image 11/200: 000010.png
Processing image 12/200: 000011.png
Processing image 13/200: 000012.png
Processing image 14/200: 000013.png
Processing image 15/200: 000014.png
Processing image 16/200: 000015.png
Processing image 17/200: 000016.png
Processing image 18/200: 000017.png
Processing image 19/200: 000018.png
Processing image 20/200: 000019.png
Processing image 21/200: 000020.png
Processing image 22/200: 000021.png
Processing image 23/200: 000022.png
Processing image 24/

'video/n60.mp4'

## Converting from raw CSV for n=60

In [ ]:
# Load only first 100 rows of CSV
df = pd.read_csv("data/raw_sets_n60.csv")

# Create a list of lists where each row is a list of pairs [a-b]
all_pairs = []
for index, row in df.iterrows():
    pairs = []
    for col in df.columns:  # Skip the first column (Pair No.)
        pair_str = row[col]
        if pd.isna(pair_str):
            continue  # Skip empty cells
        pairs.append(parse_pair(pair_str))
    all_pairs.append(pairs)

all_pairs[0]

In [ ]:
len(all_pairs)

In [ ]:
write_to_csv(all_pairs, output_csv="data/test_sets_n60.csv")